In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
# Importing face detectors
protoPath = os.path.sep.join([r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection','deploys.prototxt'])
weightsPath = os.path.sep.join([r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
net = cv2.dnn.readNet(protoPath,weightsPath)

In [4]:
net

<dnn_Net 00000162E40F7B30>

In [5]:
model = load_model(r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection\my_weights.h5')

In [17]:
image = cv2.imread(r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection\demo03.jpg')# takes an image and converts into array
image

array([[[166, 171, 180],
        [166, 171, 180],
        [166, 171, 180],
        ...,
        [193, 192, 194],
        [194, 193, 195],
        [194, 193, 195]],

       [[167, 172, 181],
        [167, 172, 181],
        [167, 172, 181],
        ...,
        [191, 190, 192],
        [192, 191, 193],
        [193, 192, 194]],

       [[167, 172, 181],
        [167, 172, 181],
        [167, 172, 181],
        ...,
        [192, 191, 193],
        [193, 192, 194],
        [194, 193, 195]],

       ...,

       [[228, 228, 228],
        [223, 223, 223],
        [222, 222, 222],
        ...,
        [219, 219, 219],
        [219, 219, 219],
        [219, 219, 219]],

       [[217, 217, 217],
        [229, 229, 229],
        [226, 226, 226],
        ...,
        [216, 216, 216],
        [216, 216, 216],
        [217, 217, 217]],

       [[217, 217, 217],
        [229, 229, 229],
        [226, 226, 226],
        ...,
        [216, 216, 216],
        [216, 216, 216],
        [217, 217, 217]]

In [18]:
image.shape

(650, 797, 3)

In [19]:
(h,w)=image.shape[:2]# taking the height and width

In [20]:
(h,w)

(650, 797)

In [21]:
blob = cv2.dnn.blobFromImage(image, 1.0,(300,300),(104.0,177.0,123.0))# resize the image (RGB only)

In [22]:
blob

array([[[[ 62.,  62.,  62., ...,  89.,  87.,  89.],
         [ 64.,  64.,  65., ...,  89.,  90.,  92.],
         [ 62.,  64.,  65., ...,  89.,  89.,  90.],
         ...,
         [121., 113., 143., ..., 115., 115., 115.],
         [122., 120., 120., ..., 115., 115., 115.],
         [123., 118., 123., ..., 113., 115., 112.]],

        [[ -6.,  -6.,  -6., ...,  15.,  13.,  15.],
         [ -4.,  -4.,  -3., ...,  15.,  16.,  18.],
         [ -6.,  -5.,  -3., ...,  15.,  15.,  16.],
         ...,
         [ 48.,  40.,  70., ...,  42.,  42.,  42.],
         [ 49.,  47.,  47., ...,  42.,  42.,  42.],
         [ 50.,  45.,  50., ...,  40.,  42.,  39.]],

        [[ 57.,  57.,  57., ...,  71.,  68.,  71.],
         [ 59.,  59.,  60., ...,  71.,  72.,  74.],
         [ 57.,  58.,  60., ...,  71.,  71.,  72.],
         ...,
         [102.,  94., 124., ...,  96.,  96.,  96.],
         [103., 101., 101., ...,  96.,  96.,  96.],
         [104.,  99., 104., ...,  94.,  96.,  93.]]]], dtype=float32)

In [23]:
blob.shape

(1, 3, 300, 300)

In [24]:
net.setInput(blob)  
detections = net.forward()# dtetcting the face

In [25]:
detections

array([[[[0.        , 1.        , 0.9999702 , ..., 0.26279536,
          0.72847646, 0.82057536],
         [0.        , 1.        , 0.11575052, ..., 4.010171  ,
          0.8216634 , 4.987138  ],
         [0.        , 1.        , 0.11499286, ..., 4.007151  ,
          4.834893  , 4.9860654 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [26]:
detections.shape

(1, 1, 200, 7)

In [28]:
for i in range(0,detections.shape[2]):
    confidence = detections[0 ,0 ,i ,2]
#     print("confidence: " ,confidence)
    if confidence> 0.5:
        # getting the x and y coordinates 
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startx , starty , endx , endy) = box.astype('int')
        (startx , starty)=(max(0 , startx) , max(0 ,starty))
        (endx , endy) = (min(w-1, endx) , min(h-1,endy))
        
        #Extract the face ROI , convert it from BRG to RGB channel , resize it to (224x224) and preprocess it
        face = image[starty:endy , startx:endx]
        face = cv2.cvtColor(face , cv2.COLOR_BGR2RGB)
        face= cv2.resize(face,(224,224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis = 0)
        
        (mask, without_mask) = model.predict(face)[0]# Display the first result that we get
        #determine the class label and color we will use to draw the bounding box and text
        label = "Mask" if mask>without_mask else "No Mask"
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        #Including the probability in label

        label ="{} : {:.2f}%".format(label , max(mask ,without_mask)*100)
        
        # Display the label and bounding boxes
        
        cv2.putText(image, label ,(startx ,starty-10) , cv2.FONT_HERSHEY_SIMPLEX , 0.45 , color,2)# diplay the text below
        cv2.rectangle(image ,(startx ,starty),(endx, endy) , color , 2)

cv2.imshow("Output " , image)
cv2.waitKey(0)
cv2.destroyAllWindows()
